In [ ]:
from os import environ, chdir

from keras.preprocessing.image import ImageDataGenerator

from keras import models, layers, optimizers, callbacks

import tensorflow as tf

import h5py

environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
chdir(r'/home/administrator/Downloads')

In [ ]:
# Setting Image and Data Generators
train_idg = ImageDataGenerator(rescale=1. / 255,
                               zoom_range=[1.0, 1.25],
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               fill_mode='reflect')

train_g = train_idg.flow_from_directory(directory=r'train1',
                                        target_size=(100, 100),
                                        class_mode='multiclass',
                                        batch_size=108,
                                        shuffle=True)

valid_idg = ImageDataGenerator(rescale=1. / 255)

valid_g = valid_idg.flow_from_directory(directory=r'valid1',
                                        target_size=(100, 100),
                                        class_mode='multiclass',
                                        batch_size=50,
                                        shuffle=True)

In [ ]:
# CNN Architecture
my_model = models.Sequential()
my_model.add(layers.Conv2D(filters=16, kernel_size=(2,2), strides=(1, 1),
                           input_shape=(100, 100, 3)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
my_model.add(layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Flatten())

my_model.add(layers.Dropout(rate=0.4))

my_model.add(layers.Dense(units=100))
my_model.add(layers.Activation('relu'))

my_model.add(layers.Dense(units=5))
my_model.add(layers.Activation('softmax'))
print(my_model.summary())

In [ ]:
# Model Loss function and Optimizer method
compile = my_model.compile(optimizer=optimizers.Adadelta(lr=0.15), loss='binary_crossentropy',
                           metrics=['accuracy'])


In [ ]:
# Settting Callbacks
chdir(r'/home/administrator/Downloads')
check_p = callbacks.ModelCheckpoint(filepath='animal_cnn_{val_acc:.2f}.h5',
                                    monitor='val_acc', verbose=1,
                                    save_best_only=True, save_weights_only=False)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3,
                                        verbose=1, cooldown=2)
callb_l = [check_p, reduce_lr]

In [ ]:
# Training Options
fit = my_model.fit_generator(generator=train_g, steps_per_epoch=22, epochs=100, verbose=1,
                             callbacks=callb_l, validation_data=valid_g, validation_steps=4)

In [ ]:
# Saving Model
my_model.save(filepath=r'animal_cnn.h5', overwrite=True)


In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs= range(len(acc))

plt.plot(epochs, acc, 'b', label='training acc')
plt.plot(epochs, val_acc, 'r', label='validation loss')
plt.title('training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='training loss')
plt.plot(epochs, val_loss, 'r', label='training loss')
plt.title('training and validation loss')
plt.legend()

plt.show()

